In [ ]:
from sklearn import cluster, decomposition, preprocessing, feature_selection
import pandas as pd
import numpy as np
from scipy.spatial import distance

In [ ]:
centers = pd.read_csv('../../data/interim/Modelling/clusterProfilesPCA.csv').iloc[:,1:]
centers

,resolutionTime,sonarCloseTime,timeInProject,fixedJiraIssues,numberCommits,refactoringExtractMethod,fixedSZZIssues,fixedSonarIssues,jiraBlocker,jiraMinor,...,commitChangeAdd,jiraTechnicalTask,codeDuplicatedLinesDensity,jiraStory,linesRemoved,codeCommentLinesDensity,commitChangeRename,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,2722.73,913.46,6.727067e+06,12.11,18.98,0.29,0.85,0.98,0.56,1.91,...,0.42,0.04,-0.01,0.14,4.39,-0.00,0.14,0.15,0.0,0.07
1,4.21,30627.87,1.988188e+08,-0.15,2231.17,157.23,190.08,324.78,0.01,-0.11,...,1.14,0.01,-0.04,0.01,11.89,0.03,1.02,0.73,0.0,-0.01
2,1410.78,2245.84,7.975709e+06,635.89,36.05,0.08,2.51,1.66,44.58,40.42,...,0.01,0.99,0.01,2.52,2.45,0.01,0.00,-0.05,-0.0,-0.05
3,36196.62,28.21,-6.119720e+03,2.99,-0.08,-0.00,0.06,0.22,0.17,1.12,...,-0.11,0.01,0.00,0.07,-1.27,0.00,-0.03,-0.04,-0.0,-0.02
4,-2.68,44072.29,1.051659e+08,0.47,316.49,6.57,18.78,40.05,-0.04,-0.17,...,1.40,-0.01,0.30,-0.02,17.04,0.15,0.50,0.45,-0.0,0.02


In [ ]:
dev = pd.read_csv('../../data/processed/DEVELOPERS_DATA.csv').iloc[:,1:]
cols = ['committer'] + list(centers.columns)
dev = dev.reindex(columns=cols)
dev.head()

,committer,resolutionTime,sonarCloseTime,timeInProject,fixedJiraIssues,numberCommits,refactoringExtractMethod,fixedSZZIssues,fixedSonarIssues,jiraBlocker,...,commitChangeAdd,jiraTechnicalTask,codeDuplicatedLinesDensity,jiraStory,linesRemoved,codeCommentLinesDensity,commitChangeRename,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,-l,0.0,22617.766389,4235880.0,0.0,27.0,0.0,0.0,0.0,0.0,...,0.111111,0.0,0.0,0.0,14.141744,0.0,0.407407,1.111111,0.0,0.0
1,1028332163,0.0,0.000000,77939.0,0.0,14.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,4.105769,0.0,0.000000,0.000000,0.0,0.0
2,A. J. David Bosschaert,0.0,0.000000,173937105.0,0.0,432.0,13.0,51.0,0.0,0.0,...,0.595349,0.0,0.0,0.0,6.916947,0.0,0.393023,0.111628,0.0,0.0
3,A195882,0.0,809.977238,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.000000,0.0,0.0,0.0,26.500000,0.0,0.000000,0.000000,0.0,0.0
4,A744013,0.0,821.388563,351970.0,0.0,5.0,0.0,0.0,4.0,0.0,...,0.250000,0.0,0.0,0.0,3.700000,0.0,1.000000,0.000000,0.0,0.0


In [ ]:
dev2 = dev.iloc[:,1:]
print(dev2.shape)
dev2.head()

(2460, 94)


,resolutionTime,sonarCloseTime,timeInProject,fixedJiraIssues,numberCommits,refactoringExtractMethod,fixedSZZIssues,fixedSonarIssues,jiraBlocker,jiraMinor,...,commitChangeAdd,jiraTechnicalTask,codeDuplicatedLinesDensity,jiraStory,linesRemoved,codeCommentLinesDensity,commitChangeRename,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown
0,0.0,22617.766389,4235880.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.0,0.0,0.0,14.141744,0.0,0.407407,1.111111,0.0,0.0
1,0.0,0.000000,77939.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,4.105769,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.000000,173937105.0,0.0,432.0,13.0,51.0,0.0,0.0,0.0,...,0.595349,0.0,0.0,0.0,6.916947,0.0,0.393023,0.111628,0.0,0.0
3,0.0,809.977238,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.000000,0.0,0.0,0.0,26.500000,0.0,0.000000,0.000000,0.0,0.0
4,0.0,821.388563,351970.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,...,0.250000,0.0,0.0,0.0,3.700000,0.0,1.000000,0.000000,0.0,0.0


Assign to each developer a label of its cluster

In [ ]:
kmeans = cluster.KMeans(n_clusters=5, init=centers, n_init=1, max_iter=1).fit(dev2)
kmeans.cluster_centers = np.array(centers)

In [ ]:
clusters = kmeans.predict(dev2)

In [ ]:
dev['cluster'] = clusters
dev.head()

,committer,resolutionTime,sonarCloseTime,timeInProject,fixedJiraIssues,numberCommits,refactoringExtractMethod,fixedSZZIssues,fixedSonarIssues,jiraBlocker,...,jiraTechnicalTask,codeDuplicatedLinesDensity,jiraStory,linesRemoved,codeCommentLinesDensity,commitChangeRename,commitChangeDelete,jiraDependencyUpgrade,commitChangeUnknown,cluster
0,-l,0.0,22617.766389,4235880.0,0.0,27.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,14.141744,0.0,0.407407,1.111111,0.0,0.0,0
1,1028332163,0.0,0.000000,77939.0,0.0,14.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.105769,0.0,0.000000,0.000000,0.0,0.0,3
2,A. J. David Bosschaert,0.0,0.000000,173937105.0,0.0,432.0,13.0,51.0,0.0,0.0,...,0.0,0.0,0.0,6.916947,0.0,0.393023,0.111628,0.0,0.0,1
3,A195882,0.0,809.977238,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,26.500000,0.0,0.000000,0.000000,0.0,0.0,3
4,A744013,0.0,821.388563,351970.0,0.0,5.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,3.700000,0.0,1.000000,0.000000,0.0,0.0,3


Within cluster variance

In [ ]:
def WCV(dev, centers):
  WCV = np.zeros(5)
  for i in range(5):  # clusters
    X = dev[dev.cluster==i].iloc[:,1:-1]
    c = [np.array(centers)[i]]
    d = distance.cdist(X, c)
    WCV[i] = d.sum()/d.shape[0]
  return [WCV, WCV.sum()]

In [ ]:
cluster, total = WCV(dev, centers)
print(cluster)
print(total)

[ 3925578.19840709 69200149.20321019 27878798.59658392   104045.29851444
 23896955.73317381]
125005527.02988943


Between cluster variance

In [ ]:
def BCV(dev, centers):
  BCV = np.zeros(5)
  x = [np.array(dev.iloc[:,1:-1].mean())]
  for i in range(5):
    n = dev[dev.cluster==i].shape[0]
    c = [np.array(centers)[i]]
    d = distance.cdist(c, x)
    BCV[i] = n*d.sum()
  return [BCV, BCV.sum()]

In [ ]:
cluster, total = BCV(dev, centers)
print(cluster)
print(total)

[1.62573161e+09 9.52231606e+09 2.21902489e+09 3.02740367e+10
 1.50556264e+10]
58696735746.71078


# **Davies–Bouldin index:**

In [ ]:
def DB(dev, centers):
  wcv, _ = WCV(dev, centers) # mean distance of all elements in cluster to centroid
  DBC = np.zeros((5,5)) # distance between centroids
  DavisBouldin = 0
  for i in range(5):  # clusters
    max = 0
    for j in range(5):
      ci = [np.array(centers)[i]]
      cj = [np.array(centers)[j]]
      d = distance.cdist(ci, cj)
      DBC[i,j] = d.sum()

      if i != j:
        val = (wcv[i]+wcv[j])/DBC[i,j]
        if val > max:
          max = val
    DavisBouldin += max
  return DavisBouldin/5

In [ ]:
DavisBouldinIndex = DB(dev, centers)
DavisBouldinIndex

11.251634654227068